In [3]:
import pandas as pd 
import seaborn as sns 
import matplotlib as plt 
%matplotlib inline
import glob
import csv
import tarfile
from tqdm import tqdm
import requests as rq
import json
from bs4 import BeautifulSoup as bs
import urllib3

In [4]:
files = glob.glob('../BL_Work/openPrescribe/uncompressed/*.tar.gz')

In [14]:
def readChem(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('CHEM') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df

In [6]:
testurl = "https://www.drugbank.ca/unearth/q?utf8=✓&searcher=drugs&query=Loperamide+Hydrochloride"
response = rq.get(testurl)
soup = bs(response.text,'html.parser')

In [7]:
A = [i for i in soup.find_all('dt') if 'Ass' in i.text]

In [8]:
dts = enumerate(soup.find('div', class_='card-content').findAll('dt'))
dds = soup.find('div', class_='card-content').findAll('dd')
dtInd = [k for (k,i) in dts if i.text == 'Associated Conditions' ][0]


In [9]:
[{i.attrs.get('href'): i.text} for i in dds[dtInd].findAll('a')]

[{u'/indications/DBCOND0089496': u'Chronic Functional Diarrhea'},
 {u'/indications/DBCOND0024414': u'Intestinal stoma leak'},
 {u'/indications/DBCOND0033890': u"Travelers' Diarrhea"},
 {u'/indications/DBCOND0102291': u'Severe or persistent diarrhea'}]

In [10]:
drugContainers = soup.find_all('div', class_ = 'unearth-search-results unearth-indication-search-results')

In [15]:
chem = readChem('201903',files)

In [16]:
chem.head()

,0,1
0,0104020D0,Codeine Phosphate Compound Mixtures
1,0104020H0,Co-Phenotrope (Diphenox HCl/Atrop Sulph)
2,0104020L0,Loperamide Hydrochloride
3,0104020N0,Opium & Morphine
4,0104020P0,Loperamide Hydrochloride & Simeticone


In [28]:
import time
urlhead = "https://www.drugbank.ca/unearth/q?utf8=✓&searcher=drugs&query="

drug_conditions_mapping = {}

for index, row in chem.iterrows():
    try:
        drug = row[1].strip().replace(' ', '+')
    except:
        print "Found corrupt drug name"
        continue
    print "Working with %s"%drug
    url = urlhead+drug
    response = rq.get(url)
    time.sleep(0.1)
    soup = bs(response.text,'html.parser')
    try:
        A = [i for i in soup.find_all('dt') if 'Ass' in i.text]
        dts = enumerate(soup.find('div', class_='card-content').findAll('dt'))
        dds = soup.find('div', class_='card-content').findAll('dd')
        dtInd = [k for (k,i) in dts if i.text == 'Associated Conditions' ][0]
        associations = [{i.attrs.get('href'): i.text} for i in dds[dtInd].findAll('a')]
        drug_conditions_mapping[row[0]]={}
        drug_conditions_mapping[row[0]]['Name'] = row[1]
        drug_conditions_mapping[row[0]]['Conditions'] = associations
    except:
        print "Drug %s was not found"%drug
    

Working with Codeine+Phosphate+Compound+Mixtures
Drug Codeine+Phosphate+Compound+Mixtures was not found
Working with Co-Phenotrope+(Diphenox+HCl/Atrop+Sulph)
Drug Co-Phenotrope+(Diphenox+HCl/Atrop+Sulph) was not found
Working with Loperamide+Hydrochloride
Working with Opium+&+Morphine
Drug Opium+&+Morphine was not found
Working with Loperamide+Hydrochloride+&+Simeticone
Working with Racecadotril
Drug Racecadotril was not found
Working with Telotristat+Ethyl
Working with Other+Chronic+Bowel+Disorder+Preps
Drug Other+Chronic+Bowel+Disorder+Preps was not found
Working with Aminosalicylic+Acid
Working with Mesalazine+(Systemic)
Drug Mesalazine+(Systemic) was not found
Working with Olsalazine+Sodium
Working with Balsalazide+Sodium
Drug Balsalazide+Sodium was not found
Working with Sulfasalazine
Working with Budesonide
Working with Hydrocortisone+Acetate
Working with Hydrocortisone
Working with Prednisolone+Sodium+Metasulphobenzoate
Drug Prednisolone+Sodium+Metasulphobenzoate was not found
W

Drug Triamterene+With+Thiazides was not found
Working with Isosorbide
Drug Isosorbide was not found
Working with Mannitol
Working with Bendroflumethiazide/Potassium
Drug Bendroflumethiazide/Potassium was not found
Working with Bumetanide/Potassium
Drug Bumetanide/Potassium was not found
Working with Bumetanide/Amiloride+Hydrochloride
Drug Bumetanide/Amiloride+Hydrochloride was not found
Working with Chlortalidone/Potassium
Drug Chlortalidone/Potassium was not found
Working with Clopamide/Potassium
Drug Clopamide/Potassium was not found
Working with Cyclopenthiazide/Potassium
Drug Cyclopenthiazide/Potassium was not found
Working with Cyclopenthiazide/Amiloride+Hydrochloride
Drug Cyclopenthiazide/Amiloride+Hydrochloride was not found
Working with Furosemide/Potassium
Drug Furosemide/Potassium was not found
Working with Hydrochlorothiazide/Potassium
Drug Hydrochlorothiazide/Potassium was not found
Working with Adenosine
Working with Amiodarone+Hydrochloride
Working with Bretylium+Tosilate

Drug Valsartan+with+Diuretic was not found
Working with Olmesartan+Medoxomil/Hydrochlorothiazide
Drug Olmesartan+Medoxomil/Hydrochlorothiazide was not found
Working with Aliskiren
Working with Trimetaphan+Camsilate
Drug Trimetaphan+Camsilate was not found
Working with Ketanserin
Drug Ketanserin was not found
Working with Amyl+Nitrite
Drug Amyl+Nitrite was not found
Working with Glyceryl+Trinitrate
Working with Isosorbide+Dinitrate
Working with Isosorbide+Mononitrate
Working with Pentaerithritol+Tetranitrate
Drug Pentaerithritol+Tetranitrate was not found
Working with Isosorbide+Mononitrate+with+Aspirin
Drug Isosorbide+Mononitrate+with+Aspirin was not found
Working with Amlodipine
Working with Trimetazidine+Hydrochloride
Drug Trimetazidine+Hydrochloride was not found
Working with Diltiazem+Hydrochloride
Working with Diltiazem+Hydrochloride+with+Thiazides
Drug Diltiazem+Hydrochloride+with+Thiazides was not found
Working with Felodipine
Working with Isradipine
Working with Lacidipine
Drug

Working with Alirocumab
Working with Fenofibrate/Simvastatin
Drug Fenofibrate/Simvastatin was not found
Working with Atorvastatin
Working with Cerivastatin
Drug Cerivastatin was not found
Working with Bezafibrate
Working with Colestyramine
Working with Chondroitin++Sulphate+A
Drug Chondroitin++Sulphate+A was not found
Working with Clofibrate
Drug Clofibrate was not found
Working with Ispaghula+Husk
Drug Ispaghula+Husk was not found
Working with Colestipol+Hydrochloride
Working with Ezetimibe
Working with Fluvastatin+Sodium
Working with Dextrothyroxine+Sodium
Drug Dextrothyroxine+Sodium was not found
Working with Fenofibrate
Working with Gemfibrozil
Working with Lovastatin
Working with Nicofuranose
Drug Nicofuranose was not found
Working with Nicotinic+Acid
Working with Omega-3+Marine+Triglycerides
Drug Omega-3+Marine+Triglycerides was not found
Working with Probucol
Drug Probucol was not found
Working with Pravastatin+Sodium
Working with Simvastatin
Working with Sitosterol
Drug Sitoste

Working with Tyloxapol
Drug Tyloxapol was not found
Working with Erdosteine
Drug Erdosteine was not found
Working with Mannitol
Working with Ivacaftor
Working with Lumacaftor/Ivacaftor
Drug Lumacaftor/Ivacaftor was not found
Working with Tezacaftor/Ivacaftor
Drug Tezacaftor/Ivacaftor was not found
Working with Other+Aromatic+Inhalation+Preps
Drug Other+Aromatic+Inhalation+Preps was not found
Working with Benzoin
Drug Benzoin was not found
Working with Menthol
Working with Pine
Drug Pine was not found
Working with Codeine+Phosphate
Working with Dextromethorphan+Hydrobromide
Working with Diamorphine+Hydrochloride
Drug Diamorphine+Hydrochloride was not found
Working with Isoaminile+Citrate
Drug Isoaminile+Citrate was not found
Working with Methadone+Hydrochloride
Working with Noscapine
Drug Noscapine was not found
Working with Pholcodine
Drug Pholcodine was not found
Working with Ephedrine+HCl/Dextrometh+Hydrob
Drug Ephedrine+HCl/Dextrometh+Hydrob was not found
Working with Other+Expector

...

Working with Ammoniated+Mercury
Drug Ammoniated+Mercury was not found
Working with Tars
Drug Tars was not found
Working with Calcipotriol
Working with Dithranol
Drug Dithranol was not found
Working with Coal+Tar+&+Corticosteroid+Preparations
Working with Dithranol+Triacetate
Drug Dithranol+Triacetate was not found
Working with Etretinate
Drug Etretinate was not found
Working with Methoxsalen
Working with 5-Methoxypsoralen
Drug 5-Methoxypsoralen was not found
Working with Mercuric+Nitrate
Drug Mercuric+Nitrate was not found
Working with Coal+Tar+Bath+Preparations
Drug Coal+Tar+Bath+Preparations was not found
Working with Pyrogallol
Drug Pyrogallol was not found
Working with Tacalcitol
Drug Tacalcitol was not found
Working with Salicylic+Acid
Working with Tazarotene
Working with Trioxsalen
Drug Trioxsalen was not found
Working with Cocois
Drug Cocois was not found
Working with Calcitriol
Working with Fumaric+Acid
Drug Fumaric+Acid was not found
Working with Cholesterol/Simvastatin
Drug C

Working with Phenothrin
Drug Phenothrin was not found
Working with Tiabendazole
Drug Tiabendazole was not found
Working with Dimeticone+(Parasiticidal)
Drug Dimeticone+(Parasiticidal) was not found
Working with Other+Minor+Cut+&+Abrasion+Preps
Drug Other+Minor+Cut+&+Abrasion+Preps was not found
Working with Aluminium+Sulfate
Drug Aluminium+Sulfate was not found
Working with Boric+Acid
Drug Boric+Acid was not found
Working with Cetrimide
Drug Cetrimide was not found
Working with Enbucrilate
Drug Enbucrilate was not found
Working with Hydrogen+Peroxide
Drug Hydrogen+Peroxide was not found
Working with Chlorhexidine+Gluconate
Drug Chlorhexidine+Gluconate was not found
Working with Dibrompropamidine+Isetionate
Drug Dibrompropamidine+Isetionate was not found
Working with Chlorquinaldol
Working with Magnesium+Sulfate
Working with Phenol
Working with Propamidine+Isetionate
Drug Propamidine+Isetionate was not found
Working with Polynoxylin
Drug Polynoxylin was not found
Working with Povidone-I

Drug Meningococcal+A+++C+Vaccine was not found
Working with Meningococcal+C+Vaccine
Drug Meningococcal+C+Vaccine was not found
Working with Meningococcal+B+Vaccine
Drug Meningococcal+B+Vaccine was not found
Working with Japanese+Encephalitis+Vaccine+(inact+ad)
Drug Japanese+Encephalitis+Vaccine+(inact+ad) was not found
Working with Japanese+Encephalitis+Vaccine+(Imported)
Drug Japanese+Encephalitis+Vaccine+(Imported) was not found
Working with Catarrh
Drug Catarrh was not found
Working with Bacillus+Calmette-Guerin+(B.C.G.)
Drug Bacillus+Calmette-Guerin+(B.C.G.) was not found
Working with Plague
Drug Plague was not found
Working with Cholera
Drug Cholera was not found
Working with Diphtheria
Drug Diphtheria was not found
Working with Hepatitis+B
Drug Hepatitis+B was not found
Working with Influenza
Drug Influenza was not found
Working with Measles
Drug Measles was not found
Working with Mumps
Drug Mumps was not found
Working with Pertussis
Drug Pertussis was not found
Working with Pneu

Drug Other+Acid+Preps was not found
Working with Other+Concentrated+Water+Preps
Drug Other+Concentrated+Water+Preps was not found
Working with Other+Essence+Preps
Drug Other+Essence+Preps was not found
Working with Other+Extract+Preps
Drug Other+Extract+Preps was not found
Working with Other+Infusion+Preps
Drug Other+Infusion+Preps was not found
Working with Other+Oil+Preps
Drug Other+Oil+Preps was not found
Working with Other+Tincture+Preps
Drug Other+Tincture+Preps was not found
Working with Other+Syrup+Preps
Drug Other+Syrup+Preps was not found
Working with Other+Base/Dil/SuspAgent/StabiliserPreps
Drug Other+Base/Dil/SuspAgent/StabiliserPreps was not found
Working with Other+Colour++Flavour+&+Sweetening+Preps
Drug Other+Colour++Flavour+&+Sweetening+Preps was not found
Working with Other+Disinfect+Preserv&SterilisingPreps
Drug Other+Disinfect+Preserv&SterilisingPreps was not found
Working with Other+Polysorbate+&+Tween+Preps
Drug Other+Polysorbate+&+Tween+Preps was not found
Working 

Drug Belts was not found
Working with Colostomy+Bags
Drug Colostomy+Bags was not found
Working with Colostomy+Sets
Drug Colostomy+Sets was not found
Working with Deodorants
Drug Deodorants was not found
Working with Discharge+Solidifying+Agents
Drug Discharge+Solidifying+Agents was not found
Working with Filters/Bridges
Drug Filters/Bridges was not found
Working with Flanges
Drug Flanges was not found
Working with Ileostomy+Bags
Drug Ileostomy+Bags was not found
Working with Ileostomy+Sets
Drug Ileostomy+Sets was not found
Working with Irrigation+Washout+Appliances
Drug Irrigation+Washout+Appliances was not found
Working with Pressure+Plates/Shields
Drug Pressure+Plates/Shields was not found
Working with Skin+Fillers+And+Protectives
Drug Skin+Fillers+And+Protectives was not found
Working with Skin+Protectors
Drug Skin+Protectors was not found
Working with Stoma+Caps/Dressings
Drug Stoma+Caps/Dressings was not found
Working with Tubing+&+Accessories
Drug Tubing+&+Accessories was not fou

In [29]:
len(drug_conditions_mapping.keys())

1624

In [30]:
%store drug_conditions_mapping

Stored 'drug_conditions_mapping' (dict)


In [33]:
# json.dump(drug_conditions_mapping,open('drug_conditions_mapping.json','wb'))